Importing libraries

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

d:\FY Projects\anime\anienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [3]:
import os
from langchain.chat_models import init_chat_model
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re known for mimicking human speech, right? But why? I guess it has something to do with their social behavior. Maybe they learn to talk to communicate with humans the way they would with other parrots. But how exactly?\n\nFirst, I remember that parrots are highly intelligent birds. Their brains are complex, and they have a good memory. So, their ability to mimic might be linked to their intelligence. But what\'s the evolutionary reason for this? In the wild, do parrots talk to each other? I think some parrots do make vocalizations to communicate with their flock or mates. So maybe talking is an extension of that natural behavior.\n\nWhen they\'re with humans, they might imitate human speech to fit into their social environment. Since they\'re social animals, they want to interact with their human companions. Mimicking speech could help them bond or get

DATA INGESTION PIPLINE 

DATA LOADING

In [4]:
class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv


SPLIT AND STORE THE DATA

In [5]:
class VectorStoreBuilder:
    def __init__(self, csv_path: str, persist_dir: str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=20
        )
        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks,
            self.embedding,
            persist_directory=self.persist_dir
        )
        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )


In [6]:
print("Building vector database...")

original_csv = "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

loader = AnimeDataLoader(original_csv, processed_csv)
processed_csv_path = loader.load_and_process()

vector_builder = VectorStoreBuilder(
    processed_csv_path,
    persist_dir=persist_dir
)
vector_builder.build_and_save_vectorstore()

print("Vector database ready.")


Building vector database...


FileNotFoundError: [Errno 2] No such file or directory: 'data/anime_with_synopsis.csv'

retriver making

In [ ]:
vector_builder = VectorStoreBuilder(
    csv_path="",
    persist_dir="chroma_db"
)

retriever = vector_builder.load_vector_store().as_retriever()


C:\Users\djadh\AppData\Local\Temp\ipykernel_5676\3258009850.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(


convert retrviver into toool

In [ ]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriever.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


initilize model

In [ ]:
model = init_chat_model("groq:qwen/qwen3-32b")
model_with_tools = model.bind_tools([anime_retriever_tool])

In [ ]:
messages = [
    {"role": "user", "content": "Can you suggest me some anime like Naruto?"}
]

# Step 1: Model decides tool usage
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools
for tool_call in ai_msg.tool_calls:
    tool_result = anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)


Here are some anime similar to **Naruto**, along with their genres and key themes:

---

### 1. **Naruto**  
   - **Genres**: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen  
   - **Why it's similar**: A classic ninja story with intense battles, personal growth, and a quest for recognition. Perfect if you love shounen action and underdog narratives.

---

### 2. **Ninin ga Shinobuden**  
   - **Genres**: Comedy, Parody, Martial Arts  
   - **Why it's similar**: A lighthearted, humorous take on the ninja world. While not as serious as Naruto, it mixes ninja tropes with absurd comedy (e.g., stealing panties to pass exams). Great for a laugh if you enjoy the ninja setting.

---

### 3. **Tsubasa Chronicle**  
   - **Genres**: Action, Adventure, Fantasy, Magic, Supernatural, Shounen  
   - **Why it's similar**: Focuses on epic quests and dimension-hopping adventures. Like Naruto, it blends action with a sense of urgency to save someone (Sakura) and restore memories. It’s mor